# Data Preparation

This notebook sets the foundation for testing the **Low-Entropy Conjecture** across three typologically different languages. We extract and clean Wikipedia corpora for **Turkish** (agglutinative), **Russian** (fusional), and **English** (analytic) to create the balanced datasets needed for our further cross-linguistic comparison.

## Corpus Extraction and Initial Processing

We begin with Wikipedia dumps from October 2018, chosen for their comparable coverage and linguistic diversity across all three languages. Wikipedia provides ideal data because it contains both frequent everyday vocabulary and specialized terminology that showcases each language's full morphological richness.

The extraction process balances thoroughness with computational efficiency. Rather than processing entire dumps, we target approximately **350,000 articles per language** to ensure sufficient lexical coverage while maintaining manageable processing times.

In [2]:
import bz2
import re
import os
import json
import pickle
from pathlib import Path
from collections import Counter
import html
from tqdm import tqdm

let's define the functions so that all works for russian, english and turkish

In [4]:
def open_corpus_file(filepath):
    if filepath.endswith('.bz2'):
        return bz2.open(filepath, 'rt', encoding='utf-8')
    else:
        return open(filepath, 'r', encoding='utf-8')

def clean_xml_entities(text):
    text = text.replace('&lt;', '<')
    text = text.replace('&gt;', '>')
    text = text.replace('&quot;', '"')
    text = text.replace('&apos;', "'")
    text = text.replace('&amp;', '&')  # This must be last
    return text

def extract_plain_text_from_xml(filepath, skip_disambiguation=True, skip_redirects=True, max_articles=1000000):
    print(f"Extracting plain text from {filepath} (max {max_articles:,} articles)...")
    
    all_text = []
    article_count = 0
    filtered_disambig = 0
    filtered_redirects = 0

    with open_corpus_file(filepath) as file:
        current_article = ""
        in_article = False
        
        for line_num, line in enumerate(tqdm(file, desc="Processing XML")):
            if '<article name="' in line:
                name_match = re.search(r'<article name="([^"]+)">', line)
                if name_match:
                    article_name = name_match.group(1)
                    current_article = line
                    in_article = True
            
            elif '</article>' in line and in_article:
                current_article += line
                article_text = current_article
                article_text = article_text.replace('\n', '&newline;')
                
                if skip_disambiguation and '<disambiguation/>' in article_text:
                    filtered_disambig += 1
                    in_article = False
                    current_article = ""
                    continue
                
                if skip_redirects and '<redirect name=' in article_text and '<content>' not in article_text:
                    filtered_redirects += 1
                    in_article = False
                    current_article = ""
                    continue
                
                article_text = re.sub(r'<!--.+?-->', '', article_text)
                article_text = re.sub(r'</?cell>', ' ', article_text)
                article_text = re.sub(r'</?content>', '', article_text)
                article_text = re.sub(r'</?wikipedia>', '', article_text)
                article_text = re.sub(r'<wikipedia lang="[^"]*">', '', article_text)
                article_text = re.sub(r'<redirect name="[^"]*"/>', '', article_text)
                article_text = re.sub(r'<links_in name="[^"]*"/>', '', article_text)
                article_text = re.sub(r'<links_out name="[^"]*"/>', '', article_text)
                article_text = re.sub(r'<category name="[^"]*"/>', '', article_text)
                article_text = re.sub(r'<crosslanguage_link language="[^"]*" name="[^"]*"/>', '', article_text)
                article_text = re.sub(r'<link target="[^"]*">', '', article_text)
                article_text = re.sub(r'</link>', '', article_text)
                article_text = re.sub(r'<textlink name="[^"]*" freq="[0-9]+"/>', '', article_text)
                article_text = re.sub(r'<disambiguation/>', '', article_text)
                article_text = re.sub(r'<article name="[^"]*">', '', article_text)
                article_text = re.sub(r'</article>', '', article_text)
                article_text = re.sub(r'</?p>', '', article_text)
                article_text = re.sub(r'</?h>', '', article_text)
                article_text = re.sub(r'</?math>', '', article_text)
                article_text = re.sub(r'</?table>', '', article_text)
                
                article_text = clean_xml_entities(article_text)
                
                article_text = re.sub(r'&newline;\s+', '&newline;', article_text)
                article_text = re.sub(r'\s+&newline;', '&newline;', article_text)
                article_text = re.sub(r'(&newline;)+', '&newline;', article_text)
                article_text = re.sub(r'\s+', ' ', article_text)
                
                article_text = article_text.replace('&newline;', '\n')
                
                clean_text = article_text.strip()
                if len(clean_text) > 50:
                    all_text.append(clean_text)
                    article_count += 1
                    
                    if article_count % 1000 == 0:
                        print(f"Processed {article_count} articles...")
                    
                    if article_count >= max_articles:
                        print(f"Reached maximum articles limit: {max_articles:,}")
                        break
                
                in_article = False
                current_article = ""
                
            elif in_article:
                current_article += line
            
            if article_count >= max_articles:
                break
    
    print(f"Extracted {article_count} articles")
    if skip_disambiguation:
        print(f"Filtered {filtered_disambig} disambiguation articles")
    if skip_redirects:
        print(f"Filtered {filtered_redirects} redirect-only articles")
    
    return all_text

def extract_text_for_language(filepath, language_name, max_articles=1000000):
    print(f"\n=== Extracting {language_name} Text (max {max_articles:,} articles) ===")
    
    article_texts = extract_plain_text_from_xml(filepath, max_articles=max_articles)
    
    combined_text = '\n\n'.join(article_texts)
    
    print(f"Total text length: {len(combined_text):,} characters")
    print(f"Number of articles: {len(article_texts):,}")
    
    if combined_text:
        sample = combined_text[:500].replace('\n', ' ')
        print(f"Sample text: {sample}...")
    
    return combined_text

def save_extracted_text(text, language_code, output_dir="../data/processed/raw_text"):
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    output_file = Path(output_dir) / f"{language_code}_text.txt"
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(text)
    
    print(f"Saved text to: {output_file}")
    print(f"File size: {output_file.stat().st_size / (1024*1024):.1f} MB")

def create_vocabulary_from_text(text, language_code, top_k=10000, min_word_len=2):
    print(f"\nCreating vocabulary for {language_code}...")
    
    words = text.lower().split()
    filtered_words = []
    for word in words:
        word = re.sub(r'^[^\w]+|[^\w]+$', '', word, flags=re.UNICODE)
        if len(word) >= min_word_len and re.search(r'[a-zA-Zа-яё\u00C0-\u017F\u0100-\u024F\u1E00-\u1EFF]', word):
            filtered_words.append(word)
    
    from collections import Counter
    word_counts = Counter(filtered_words)
    
    top_words = word_counts.most_common(top_k)
    vocabulary = [word for word, count in top_words]
    
    print(f"Total words: {len(filtered_words):,}")
    print(f"Unique words: {len(word_counts):,}")
    print(f"Top {top_k} vocabulary: {len(vocabulary):,}")
    print(f"Sample vocabulary: {vocabulary[:10]}")
    
    return vocabulary

def extract_turkish(max_articles=350000):
    return extract_text_for_language('../data/raw_corpora/trwiki-20181001-corpus.xml.bz2', 'Turkish', max_articles)

def extract_russian(max_articles=350000):
    return extract_text_for_language('../data/raw_corpora/ruwiki-20181001-corpus.xml.bz2', 'Russian', max_articles)

def extract_english(max_articles=350000):
    return extract_text_for_language('../data/raw_corpora/enwiki-20181001-corpus.xml.bz2', 'English', max_articles)

def analyze_text_structure(text, language_name):
    print(f"\n=== Text Structure Analysis for {language_name} ===")
    
    total_chars = len(text)
    lines = text.split('\n')
    non_empty_lines = [line for line in lines if line.strip()]
    
    print(f"Total characters: {total_chars:,}")
    print(f"Total lines: {len(lines):,}")
    print(f"Non-empty lines: {len(non_empty_lines):,}")
    print(f"Average characters per line: {total_chars / len(non_empty_lines):.1f}")
    
    words = text.split()
    print(f"Total words: {len(words):,}")
    print(f"Average words per line: {len(words) / len(non_empty_lines):.1f}")
    
    double_newlines = text.count('\n\n')
    print(f"Article separators (\\n\\n): {double_newlines:,}")
    print(f"Estimated articles: {double_newlines + 1:,}")
    
    print(f"\nStructure: Text contains full articles separated by double newlines")
    print(f"Each article contains multiple sentences in paragraph form")
    print(f"Words are separated by spaces (not pre-tokenized)")

In [5]:
turkish_text = extract_turkish()


=== Extracting Turkish Text (max 350,000 articles) ===
Extracting plain text from trwiki-20181001-corpus.xml.bz2 (max 350,000 articles)...


Processing XML: 238564it [00:00, 399321.30it/s]

Processed 1000 articles...


Processing XML: 401105it [00:01, 222504.70it/s]

Processed 2000 articles...


...

Processed 346000 articles...


Processing XML: 19473236it [01:22, 333380.51it/s]

Processed 347000 articles...
Processed 348000 articles...


Processing XML: 19568769it [01:22, 237023.77it/s]

Processed 349000 articles...
Processed 350000 articles...
Reached maximum articles limit: 350,000
Extracted 350000 articles
Filtered 4789 disambiguation articles
Filtered 0 redirect-only articles
Total text length: 587,824,356 characters
Number of articles: 350,000
Sample text: Cengiz Han (Cenghis Khan, Çinggis Haan ya da doğum adıyla Temuçin (anlamı: demirci), Moğolca: Чингис Хаан ya da "Tengiz" (anlamı: deniz), ; d. 1162 – ö. 18 Ağustos 1227), Moğol komutan, hükümdar ve Moğol İmparatorluğu'nun kurucusudur. Cengiz Han, 13. Yüzyılın başında Orta Asya'daki tüm göçebe bozkır kavimlerini birleştirerek bir ulus haline getirdi ve o ulusu Moğol siyasi kimliği çatısı altında topladı. Dünya tarihinin en büyük askeri dehalarından biri olarak kabul edilen Cengiz Han, hükümdarlığ...


In [6]:
russian_text = extract_russian()


=== Extracting Russian Text (max 350,000 articles) ===
Extracting plain text from ruwiki-20181001-corpus.xml.bz2 (max 350,000 articles)...


Processing XML: 243993it [00:01, 135577.82it/s]

Processed 1000 articles...


Processing XML: 477701it [00:03, 173210.25it/s]

Processed 2000 articles...


...

Processed 347000 articles...
Processed 348000 articles...


Processing XML: 23814028it [02:33, 220632.80it/s]

Processed 349000 articles...


Processing XML: 23852945it [02:34, 154629.11it/s]


Processed 350000 articles...
Reached maximum articles limit: 350,000
Extracted 350000 articles
Filtered 18118 disambiguation articles
Filtered 0 redirect-only articles
Total text length: 1,267,408,989 characters
Number of articles: 350,000
Sample text: Литва́ ( ), официальное название — Лито́вская Респу́блика ( ) — государство, расположенное в Северной Европе (одна из стран Балтии). Столица страны — Вильнюс. Площадь — 65 300 км². Протяжённость с севера на юг - 280 км, а с запада на восток - 370 км. Население составляет 3 054 000 человек. Согласно более современной гипотезе, название страны могло произойти от этнонима «леты» или «лейти», которым жители окрестных земель называли дружинников литовских князей. В начале XIII века в земли балтов-язы...


In [7]:
english_text = extract_english()


=== Extracting English Text (max 350,000 articles) ===
Extracting plain text from enwiki-20181001-corpus.xml.bz2 (max 350,000 articles)...


Processing XML: 196455it [00:01, 104854.44it/s]

Processed 1000 articles...


Processing XML: 350428it [00:03, 96977.71it/s] 

Processed 2000 articles...


...

Processed 348000 articles...


Processing XML: 33438688it [04:14, 321180.76it/s]

Processed 349000 articles...


Processing XML: 33622002it [04:15, 131654.67it/s]


Processed 350000 articles...
Reached maximum articles limit: 350,000
Extracted 350000 articles
Filtered 5080 disambiguation articles
Filtered 0 redirect-only articles
Total text length: 2,413,476,077 characters
Number of articles: 350,000
Sample text: Anarchism is a political philosophy that advocates self-governed societies based on voluntary institutions. These are often described as stateless societies, although several authors have defined them more specifically as institutions based on non-hierarchical or free associations. Anarchism holds the state to be undesirable, unnecessary and harmful. According to Peter Kropotkin, Godwin was "the first to formulate the political and economical conceptions of anarchism, even though he did not give...


## Now let's save all of that so that we don't depend on the kernel

In [8]:
from pathlib import Path

In [9]:
Path("research_data").mkdir(exist_ok=True)

In [10]:
with open("../data/processed/raw_text/turkish_text.txt", 'r', encoding='utf-8') as f:
    sample = f.read(500)
    print("Turkish file sample:", sample[:200] + "...")

Saved Turkish text: 587,824,356 characters


In [11]:
with open("../data/processed/raw_text/russian_text.txt", 'r', encoding='utf-8') as f:
    sample = f.read(500)
    print("Russian file sample:", sample[:200] + "...")

Saved Russian text: 1,267,408,989 characters


In [12]:
with open("../data/processed/raw_text/english_text.txt", 'r', encoding='utf-8') as f:
    sample = f.read(500)
    print("English file sample:", sample[:200] + "...")

Saved English text: 2,413,476,077 characters


In [13]:
import os

In [14]:
for lang in ['turkish', 'russian', 'english']:
    filepath = f"../data/processed/raw_text/{lang}_text.txt"
    if os.path.exists(filepath):
        size_mb = os.path.getsize(filepath) / (1024 * 1024)
        print(f"{lang.capitalize()} file size: {size_mb:.1f} MB")

Turkish file size: 603.2 MB
Russian file size: 2122.9 MB
English file size: 2310.5 MB


At this point we can already notice something interesting: Turkish produces the smallest amount of text -- just 603 MB despite having 350k articles, while Russian generates 2123 MB and English yields some 2311 MB of text from the same article count. 


> This already tells us that Turkish manages semantics differently due to its agglutinative morphology. Apparently this allows Turkish to pack meanings more compactly at the word level by making use of extensive suffixation, whereas the more analytic English and fusional Russian require more words to express equivalent semantic content.

In [15]:
with open("../data/processed/raw_text/russian_text.txt", 'r', encoding='utf-8') as f:
    sample = f.read(500)
    print("Russian file sample:", sample[:200] + "...")

Russian file sample: Литва́ ( ), официальное название — Лито́вская Респу́блика ( ) — государство, расположенное в Северной Европе (одна из стран Балтии). Столица страны — Вильнюс.
Площадь — 65 300 км². Протяжённость с сев...


In [16]:
with open("../data/processed/raw_text/turkish_text.txt", 'r', encoding='utf-8') as f:
    sample = f.read(500)
    print("Turkish file sample:", sample[:200] + "...")

Turkish file sample: Cengiz Han (Cenghis Khan, Çinggis Haan ya da doğum adıyla Temuçin (anlamı: demirci), Moğolca: Чингис Хаан ya da "Tengiz" (anlamı: deniz), ; d. 1162 – ö. 18 Ağustos 1227), Moğol komutan, hükümdar ve Mo...


In [17]:
with open("../data/processed/raw_text/english_text.txt", 'r', encoding='utf-8') as f:
    sample = f.read(500)
    print("English file sample:", sample[:200] + "...")

English file sample: Anarchism is a political philosophy that advocates self-governed societies based on voluntary institutions. These are often described as stateless societies, although several authors have defined them...


Next up:
clean the text in brackets like 
- (одна из стран Балтии) or
- (Cenghis Khan, Çinggis Haan ya da doğum adıyla Temuçin (anlamı: demirci), Moğolca: Чингис Хаан ya da "Tengiz" (anlamı: deniz), ; d. 1162 – ö. 18 Ağustos 1227)

Multiple sources suggest removing the parenthetical text

https://raw.githubusercontent.com/eberlitz/pt-br-corpus/master/README.md#:~:text=,will%20allow%20abbreviations%2C%20like%20%27Dr
https://willbeason.com/2021/08/06/cleaning-the-wikipedia-corpus-articles-and-text/

If we search for good wikipedia texts cleaning practices, we can find:
https://willbeason.com/2021/08/06/cleaning-the-wikipedia-corpus-articles-and-text/
where the author says:

> I'm currently debating whether to exclude parenthetical text (such as this). Simply ignoring the parentheses often breaks grammatical structure, and can sometime form their own prose-like thoughts... Update: I will be discarding parenthetical statements. They're noisy and if I don't have enough data I can care about that problem later.

## Text Cleaning and Normalization

Raw Wikipedia text contains all sorts of artifacts that could hinder morphological and semantic analyses. We should remove all that markup noise before we proceed.

We will go with the basic set of cleaning operations recommeded here: 

https://raw.githubusercontent.com/eberlitz/pt-br-corpus/master/README.md#:~:text=,will%20allow%20abbreviations%2C%20like%20%27Dr

keeping in mind that we have already done the following:
- removed the HTML/XML tags
- converted the XML entities  like `&lt;`, `&gt;`, `&quot;`, `&apos;`, `&amp;`
- removed Wikipedia markup  like links, categories and crosslanguage links
- basic whitespace normalization

In [32]:
import pandas as pd
import re
import os
from pathlib import Path
from tqdm import tqdm
import time
import warnings
warnings.filterwarnings('ignore')

Now we proceed straight to cleaning the language files. We still save the originals though

Like we noticed before, our files are huge, so looping over them multiple times and using simple regex won't do the job in any reasonable time (I didn't manage to get a single Turkish corpus to be processed after nearly 2 hours). Instead we can use **vectorized string operations** which are $10$-$50$x faster than loops and process each corpus in chunks.

In [33]:
class Patterns:
    parentheses = re.compile(r'\([^()]*\)')
    square_brackets = re.compile(r'\[[^\]]*\]')
    numbers = re.compile(r'\b\d+[,.]?\d*\b')
    measurements = re.compile(r'\b\d+\s*km²?\b')
    long_dashes = re.compile(r'[–—−‑]+')
    fancy_quotes = re.compile(r'[""„«»]')
    fancy_apostrophes = re.compile(r'[''‚`]')
    multiple_spaces = re.compile(r'\s+')
    space_punct = re.compile(r'\s+([.!?])')

In [34]:
def vectorized_clean_chunk(articles_series):
    stats = {
        'parentheses': articles_series.str.count(r'\(').sum(),
        'numbers': articles_series.str.count(r'\b\d+').sum(),
        'chars_before': articles_series.str.len().sum()
    }
    
    articles_series = articles_series.str.replace(Patterns.parentheses, '', regex=True)
    articles_series = articles_series.str.replace(Patterns.square_brackets, '', regex=True)
    articles_series = articles_series.str.replace(Patterns.numbers, '0', regex=True)
    articles_series = articles_series.str.replace(Patterns.measurements, '', regex=True)
    articles_series = articles_series.str.replace(Patterns.long_dashes, '-', regex=True)
    articles_series = articles_series.str.replace(Patterns.fancy_quotes, '"', regex=True)
    articles_series = articles_series.str.replace(Patterns.fancy_apostrophes, "'", regex=True)
    articles_series = articles_series.str.replace(Patterns.multiple_spaces, ' ', regex=True)
    articles_series = articles_series.str.replace(Patterns.space_punct, r'\1', regex=True)
    articles_series = articles_series.str.strip()
    
    articles_series = articles_series[articles_series.str.len() >= 10]
    
    stats['chars_after'] = articles_series.str.len().sum()
    stats['articles_kept'] = len(articles_series)
    
    return articles_series.tolist(), stats

def clean_language_file(input_file, output_file, chunk_size_mb=50):
    file_size_mb = os.path.getsize(input_file) / (1024 * 1024)
    chunk_size = chunk_size_mb * 1024 * 1024
    
    print(f"File: {file_size_mb:.1f} MB, Chunk size: {chunk_size_mb} MB")
    
    Path(output_file).parent.mkdir(parents=True, exist_ok=True)
    
    total_stats = {'parentheses': 0, 'numbers': 0, 'chars_before': 0, 'chars_after': 0, 'articles': 0}
    
    chunks_processed = 0
    
    with open(input_file, 'r', encoding='utf-8') as infile, \
         open(output_file, 'w', encoding='utf-8') as outfile:
        
        buffer = ""
        file_position = 0
        
        with tqdm(total=file_size_mb, unit='MB', desc="Processing") as pbar:
            
            while True:
                chunk = infile.read(chunk_size)
                if not chunk:
                    if buffer.strip():
                        articles = [art.strip() for art in buffer.split('\n\n') if art.strip()]
                        if articles:
                            cleaned_articles, chunk_stats = vectorized_clean_chunk(pd.Series(articles))
                            for article in cleaned_articles:
                                outfile.write(article + '\n\n')
                            
                            for key in ['parentheses', 'numbers', 'chars_before', 'chars_after']:
                                total_stats[key] += chunk_stats[key]
                            total_stats['articles'] += len(cleaned_articles)
                    break
                
                chunk_size_mb_actual = len(chunk.encode('utf-8')) / (1024 * 1024)
                pbar.update(chunk_size_mb_actual)
                
                full_text = buffer + chunk
                last_article_end = full_text.rfind('\n\n')
                
                if last_article_end != -1:
                    process_text = full_text[:last_article_end + 2]
                    buffer = full_text[last_article_end + 2:]
                else:
                    process_text = full_text
                    buffer = ""
                
                if process_text.strip():
                    articles = [art.strip() for art in process_text.split('\n\n') if art.strip()]
                    
                    if articles:
                        # here we are doing the vectorized processing so it's done faster
                        cleaned_articles, chunk_stats = vectorized_clean_chunk(pd.Series(articles))
                        
                        for article in cleaned_articles:
                            outfile.write(article + '\n\n')
                        
                        # Update statistics
                        for key in ['parentheses', 'numbers', 'chars_before', 'chars_after']:
                            total_stats[key] += chunk_stats[key]
                        total_stats['articles'] += len(cleaned_articles)
                        
                        chunks_processed += 1
    
    # Final statistics
    output_size_mb = os.path.getsize(output_file) / (1024 * 1024)
    reduction_pct = (1 - output_size_mb / file_size_mb) * 100
    
    print(f"Processed {chunks_processed} chunks")
    print(f"Results: {total_stats['articles']:,} articles, "
          f"{total_stats['parentheses']:,} parentheses removed, "
          f"{total_stats['numbers']:,} numbers normalized")
    print(f"Size: {file_size_mb:.1f} MB -> {output_size_mb:.1f} MB ({reduction_pct:.1f}% reduction)")
    print(f"Saved: {output_file}")
    
    return output_file

In [35]:
Path("../data/processed/clean_text").mkdir(parents=True, exist_ok=True)

In [37]:
for language in ['turkish', 'russian', 'english']:
    print(f"\nProcessing {language.title()}:")
    input_file = f"../data/processed/raw_text/{language}_text.txt"
    output_file = f"../data/processed/clean_text/{language}_clean.txt"
    
    start_time = time.time()
    result_file = clean_language_file(input_file, output_file)
    processing_time = time.time() - start_time
    
    print(f"Processing time: {processing_time:.1f} seconds")


Processing Turkish:
File: 603.2 MB, Chunk size: 50 MB


Processing: 100%|████████████████████████████████| 603.1974821090698/603.1974821090698 [00:45<00:00, 13.27MB/s]


Processed 12 chunks
Results: 349,969 articles, 1,688,200 parentheses removed, 10,673,264 numbers normalized
Size: 603.2 MB -> 560.0 MB (7.2% reduction)
Saved: research_data/clean_data/turkish_clean.txt
Processing time: 45.5 seconds

Processing Russian:
File: 2122.9 MB, Chunk size: 50 MB


Processing: 100%|████████████████████████████████| 2122.904839515686/2122.904839515686 [01:38<00:00, 21.60MB/s]


Processed 25 chunks
Results: 349,988 articles, 4,304,471 parentheses removed, 16,250,307 numbers normalized
Size: 2122.9 MB -> 1961.2 MB (7.6% reduction)
Saved: research_data/clean_data/russian_clean.txt
Processing time: 98.3 seconds

Processing English:
File: 2310.5 MB, Chunk size: 50 MB


Processing: 100%|████████████████████████████████| 2310.487558364868/2310.487558364868 [02:57<00:00, 13.03MB/s]

Processed 47 chunks
Results: 350,000 articles, 5,759,614 parentheses removed, 26,233,991 numbers normalized
Size: 2310.5 MB -> 2159.5 MB (6.5% reduction)
Saved: research_data/clean_data/english_clean.txt
Processing time: 177.4 seconds


In [38]:
import random

In [44]:
chunks = [
    c.strip()
    for c in Path("../data/processed/clean_text/russian_clean.txt")
             .read_text(encoding="utf-8")
             .split("\n\n")
    if c.strip()
]

snippet = random.choice(chunks)
print(snippet)

Марк Антуан Эду - французский переводчик. Начал свою переводческую деятельность с участия в работе над переводом с английского языка "Медицинского словаря" Роберта Джеймса. Среди последующих переводов Эду - "История поэзии" Джона Брауна , "Теория нравственных чувств" Адама Смита , путевые записки Джона Белла , философские сочинения Фрэнсиса Хатчесона и др.Примечания


In [40]:
chunks = [
    c.strip()
    for c in Path("../data/processed/clean_text/turkish_clean.txt")
             .read_text(encoding="utf-8")
             .split("\n\n")
    if c.strip()
]

snippet = random.choice(chunks)
print(snippet)

Korumalı kruvazör, 0. yüzyıl sonlarında ortaya çıkan bir savaş gemisi tipidir. Bu gemi tipinde zırh kaplaması geminin makine dairesi gibi hayati bölümleri zırhla korunur, ayrıca geminin kömür ambarları da zırha ek koruma sağlarlar. Korumalı kruvazörler, yanlarda bir zırh kemeri olan zırhlı kruvazörlere alternatif bir tasarımdı.


In [42]:
chunks = [
    c.strip()
    for c in Path("../data/processed/clean_text/english_clean.txt")
             .read_text(encoding="utf-8")
             .split("\n\n")
    if c.strip()
]

snippet = random.choice(chunks)
print(snippet)

Pierre Antoine Noël Bruno, comte Daru was a French soldier, statesman, historian, and poet. The French generally refer to him as Pierre Daru.Early career Born in Montpellier, he was educated at the Oratorian-maintained military school of Tournon, and entered artillery service at an early age. He also took an interest in literature, and he published several minor pieces, until the outbreak of the French Revolution made him concentrate on his military assignments. In 0 he became commissary to the army, protecting the coasts of Brittany from projected descents of the British, or of French Royalists. Thrown into prison during the Reign of Terror, on an unsubstantiated charge of friendliness to the Royalists and the British, he was released after the fall of Maximilien Robespierre in the summer of 0 , and rose through the ranks until, in 0, he became chief commissary to the French Revolutionary Army serving under André Masséna in the north of Switzerland. In that position he won repute for 

> Okay i think the texts have been reasonably well cleaned

## Texts are ready

Now we have cleaned corpora that we will be using for the morphological complexity measurement and semantic network construction. 

The next phase will use these corpora to extract frequency-ranked vocabulary lists and begin the morphological analysis.